In [2]:
import pandas as pd
import numpy as np
import requests
from collections import deque
from functools import reduce
import matplotlib.pyplot as plt
#pd.options.display.float_format = '{:,.0f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)

# This notebook outlines the download and formatting process for the HUD residential building permits dataset for counties and places in the GNRC region.

Go to this page: https://socds.huduser.gov/permits/  
+ Under "Main Criteria" select "States and Counties"  
+ Under "Periodicity" select "Annual"  
+ Under "Select State(s)" select "Tennessee", then select "Show Counties and Jurisdictions for Selected States"  
+ Under "Select Counties", highlight all counties, do the same for all jurisdictions under "Select Permitting Jurisdictions"  
+ Under "Select Year(s), select the most recent available year  
+ Under "Select Series", select "All Permits", "Single Family", and "All Multifamily"  
+ Finally, select the "Get Data" button  
This will populate a page with the information in table format. Scroll to the bottom and download in excel or csv format. 

Save as a csv in the "Data Downloads" folder and import as downloaded.

In [3]:
data = pd.read_csv('../../Data Downloads/HUD Building Permits.csv')

In [4]:
data.head()

,Location,Year,Series,Series Code,Permits
0,Anderson County,2021,Total Units,1,458
1,Anderson County,2021,Units in Single-Family Structures,2,261
2,Anderson County,2021,Units in All Multi-Family Structures,3,197
3,Bedford County,2021,Total Units,1,390
4,Bedford County,2021,Units in Single-Family Structures,2,390


When you examine the data download in excel, you can see that the headers repeat every few records, delete these records. 

In [7]:
data = data.loc[data['Location'] != 'Location']

In [41]:
#strip extra spaces
data['Location'] = data['Location'].str.strip()

In [8]:
data.head(100)

,Location,Year,Series,Series Code,Permits
0,Anderson County,2021,Total Units,1,458
1,Anderson County,2021,Units in Single-Family Structures,2,261
2,Anderson County,2021,Units in All Multi-Family Structures,3,197
3,Bedford County,2021,Total Units,1,390
4,Bedford County,2021,Units in Single-Family Structures,2,390
5,Bedford County,2021,Units in All Multi-Family Structures,3,0
6,Benton County,2021,Total Units,1,4
7,Benton County,2021,Units in Single-Family Structures,2,4
8,Benton County,2021,Units in All Multi-Family Structures,3,0
9,Blount County,2021,Total Units,1,922


We have three different target geographies that are downloaded: counties, unincorporated areas, and places. We can split these up to concatenate later as the data cleaning processes will be different. For now, we are only taking geographies within the GNRC 14 county operating region. You can see from the download that all counties contain the string "County", and that unincorporated areas contain the string "COUNTY".

In [9]:
counties = data[data['Location'].str.contains('County')]

In [12]:
counties.tail(100)

,Location,Year,Series,Series Code,Permits
180,Meigs County,2021,Units in All Multi-Family Structures,3,0
182,Monroe County,2021,Total Units,1,215
183,Monroe County,2021,Units in Single-Family Structures,2,215
184,Monroe County,2021,Units in All Multi-Family Structures,3,0
185,Montgomery County,2021,Total Units,1,4008
186,Montgomery County,2021,Units in Single-Family Structures,2,2119
187,Montgomery County,2021,Units in All Multi-Family Structures,3,1889
188,Moore County,2021,Total Units,1,30
189,Moore County,2021,Units in Single-Family Structures,2,30
190,Moore County,2021,Units in All Multi-Family Structures,3,0


In [14]:
region = ['Cheatham County', 'Davidson County', 'Dickson County', 'Houston County', 'Humphreys County', 'Maury County', 'Montgomery County', 
          'Robertson County', 'Rutherford County', 'Stewart County', 'Sumner County', 'Trousdale County', 'Williamson County', 'Wilson County']

In [16]:
counties = counties.loc[counties['Location'].isin(region)]

In [18]:
counties['Location'] = counties['Location'] + ", Tennessee"

C:\Users\jmccall\AppData\Local\Temp\ipykernel_9796\3502798359.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties['Location'] = counties['Location'] + ", Tennessee"


In [31]:
counties.head(100)

,Location,Year,Series,Series Code,Permits
27,"Cheatham County, Tennessee",2021,Total Units,1,407
28,"Cheatham County, Tennessee",2021,Units in Single-Family Structures,2,260
29,"Cheatham County, Tennessee",2021,Units in All Multi-Family Structures,3,147
51,"Davidson County, Tennessee",2021,Total Units,1,16310
52,"Davidson County, Tennessee",2021,Units in Single-Family Structures,2,4105
53,"Davidson County, Tennessee",2021,Units in All Multi-Family Structures,3,12205
60,"Dickson County, Tennessee",2021,Total Units,1,617
61,"Dickson County, Tennessee",2021,Units in Single-Family Structures,2,473
62,"Dickson County, Tennessee",2021,Units in All Multi-Family Structures,3,144
121,"Houston County, Tennessee",2021,Total Units,1,1


Counties are ready to go, now for Unincorporated Areas

In [42]:
unincorporated = data[data['Location'].str.contains('COUNTY')]
unincorporated.head()

,Location,Year,Series,Series Code,Permits
303,ANDERSON COUNTY UNINCORPORATED AREA,2021,Total Units,1,83
304,ANDERSON COUNTY UNINCORPORATED AREA,2021,Units in Single-Family Structures,2,83
305,ANDERSON COUNTY UNINCORPORATED AREA,2021,Units in All Multi-Family Structures,3,0
339,BEDFORD COUNTY UNINCORPORATED AREA,2021,Total Units,1,173
340,BEDFORD COUNTY UNINCORPORATED AREA,2021,Units in Single-Family Structures,2,173


In [43]:
region = ['CHEATHAM COUNTY UNINCORPORATED AREA', 'DAVIDSON COUNTY UNINCORPORATED AREA', 'DICKSON COUNTY UNINCORPORATED AREA', 
          'HOUSTON COUNTY UNINCORPORATED AREA', 'HUMPHREYS COUNTY UNINCORPORATED AREA', 'MAURY COUNTY UNINCORPORATED AREA', 
          'MONTGOMERY COUNTY UNINCORPORATED AREA', 'ROBERTSON COUNTY UNINCORPORATED AREA', 'RUTHERFORD COUNTY UNINCORPORATED AREA', 
          'STEWART COUNTY UNINCORPORATED AREA', 'SUMNER COUNTY UNINCORPORATED AREA', 'TROUSDALE COUNTY UNINCORPORATED AREA', 
          'WILLIAMSON COUNTY UNINCORPORATED AREA', 'WILSON COUNTY UNINCORPORATED AREA']

#Davidson, Houston, Humphreys nor Stewart have unincorporated lines

In [44]:
unincorporated = unincorporated.loc[unincorporated['Location'].isin(region)]

In [51]:
unincorporated['Location'] = unincorporated['Location'].replace({'CHEATHAM COUNTY UNINCORPORATED AREA': 'Cheatham Unincorporated', 
                                                                 'DICKSON COUNTY UNINCORPORATED AREA': "Dickson Unincorporated", 
                                                                 'MAURY COUNTY UNINCORPORATED AREA': 'Maury Unincorporated', 
                                                                 'MONTGOMERY COUNTY UNINCORPORATED AREA': 'Montgomery Unincorporated', 
                                                                 'ROBERTSON COUNTY UNINCORPORATED AREA': 'Robertson Unincorporated', 
                                                                 'RUTHERFORD COUNTY UNINCORPORATED AREA': 'Rutherford Unincorporated',
                                                                 'SUMNER COUNTY UNINCORPORATED AREA': 'Sumner Unincorporated', 
                                                                 'TROUSDALE COUNTY UNINCORPORATED AREA': 'Trousdale Unincorporated', 
                                                                 'WILLIAMSON COUNTY UNINCORPORATED AREA': 'Williamson Unincorporated', 
                                                                 'WILSON COUNTY UNINCORPORATED AREA': 'Wilson Unincorporated'})

In [52]:
unincorporated.head()

,Location,Year,Series,Series Code,Permits
435,Cheatham Unincorporated,2021,Total Units,1,100
436,Cheatham Unincorporated,2021,Units in Single-Family Structures,2,100
437,Cheatham Unincorporated,2021,Units in All Multi-Family Structures,3,0
528,Dickson Unincorporated,2021,Total Units,1,290
529,Dickson Unincorporated,2021,Units in Single-Family Structures,2,290


Places are going to be more difficult, we're looking for the following: 
Adams city, Tennessee: Robertson  
Ashland City town, Tennessee: Cheatham  
Belle Meade city, Tennessee: Davidson  
Berry Hill city, Tennessee: Davidson  
Brentwood city, Tennessee: Williamson  
Burns town, Tennessee: Dickson  
Cedar Hill city, Tennessee: Robertson  
Charlotte town, Tennessee: Dickson  
Clarksville city, Tennessee: Montgomery  
Columbia city, Tennessee: Maury  
Coopertown town, Tennessee: Robertson  
Cross Plains city, Tennessee: Robertson  
Cumberland City town, Tennessee: Stewart  
Dickson city, Tennessee: Dickson  
Dover city, Tennessee: Stewart  
Eagleville city, Tennessee: Rutherford  
Erin city, Tennessee: Houston  
Fairview city, Tennessee: Williamson  
Forest Hills city, Tennessee: Davidson  
Franklin city, Tennessee: Williamson  
Gallatin city, Tennessee: Sumner  
Goodlettsville city, Tennessee: Davidson/Sumner  
Greenbrier town, Tennessee: Robertson  
Hendersonville city, Tennessee: Sumner  
Kingston Springs town, Tennessee: Cheatham  
La Vergne city, Tennessee: Rutherford  
Lafayette city, Tennessee: Macon  
Lebanon city, Tennessee: Wilson  
McEwen city, Tennessee: Humphreys  
Millersville city, Tennessee: Robertson/Sumner  
Mitchellville city, Tennessee: Sumner  
Mount Juliet city, Tennessee: Wilson  
Mount Pleasant city, Tennessee: Maury  
Murfreesboro city, Tennessee: Rutherford  
Nashville-Davidson metropolitan government (balance): Davidson  
New Johnsonville city, Tennessee: Humphreys  
Nolensville town, Tennessee: Williamson  
Oak Hill city, Tennessee: Davidson  
Pegram town, Tennessee: Cheatham  
Pleasant View city, Tennessee: Cheatham  
Portland city, Tennessee: Robertson/Sumner  
Ridgetop city, Tennessee: Davidson/Robertson  
Slayden town, Tennessee: Dickson  
Smyrna town, Tennessee: Rutherford  
Spring Hill city, Tennessee: Maury/Williamson  
Springfield city, Tennessee: Robertson  
Tennessee Ridge town, Tennessee: Houston/Stewart  
Thompson's Station town, Tennessee: Williamson  
Vanleer town, Tennessee: Dickson  
Watertown city, Tennessee: Wilson  
Waverly city, Tennessee: Humphreys  
Westmoreland town, Tennessee: Sumner  
White Bluff town, Tennessee: Dickson  
White House city, Tennessee: Robertson/Sumner    

The downloaded data is in all caps, and doesn't have the ", Tennessee" of the "town.. city... etc.". There are 54 records. We can reverse index into the geographies that are *not* counties or unincorporated areas, reformat the capital letters, and then I'll make a list of only the first word of the place to run through and see how close we can get that way.


In [56]:
place = data[~data['Location'].str.contains('County')]
place = place[~place['Location'].str.contains('COUNTY')]
place.head()

,Location,Year,Series,Series Code,Permits
282,ADAMS,2021,Total Units,1,0
283,ADAMS,2021,Units in Single-Family Structures,2,0
284,ADAMS,2021,Units in All Multi-Family Structures,3,0
285,ADAMSVILLE TOWN,2021,Total Units,1,1
286,ADAMSVILLE TOWN,2021,Units in Single-Family Structures,2,1


In [71]:
place['Location'] = place['Location'].str.title()
place.head(50)

,Location,Year,Series,Series Code,Permits
282,Adams,2021,Total Units,1,0
283,Adams,2021,Units in Single-Family Structures,2,0
284,Adams,2021,Units in All Multi-Family Structures,3,0
285,Adamsville Town,2021,Total Units,1,1
286,Adamsville Town,2021,Units in Single-Family Structures,2,1
287,Adamsville Town,2021,Units in All Multi-Family Structures,3,0
288,Alamo Town,2021,Total Units,1,0
289,Alamo Town,2021,Units in Single-Family Structures,2,0
290,Alamo Town,2021,Units in All Multi-Family Structures,3,0
291,Alcoa,2021,Total Units,1,192


In [62]:
places = ["Adams", "Ashland City Town", "Belle Meade", "Berry Hill", "Brentwood", "Burns", "Cedar Hill", "Charlotte", "Clarksville", "Columbia", "Coopertown", 
          "Cross Plains", "Cumberland City", "Dickson", "Dover", "Eagleville", "Erin", "Fairview", "Forest Hills", "Franklin", "Gallatin"  , "Goodlettsville", 
          "Greenbrier", "Hendersonville", "Kingston Springs", "La Vergne", "Lafayette", "Lebanon", "McEwen", "Millersville", "Mitchellville", "Mount Juliet", 
          "Mount Pleasant", "Murfreesboro", "Nashville-Davidson", "New Johnsonville", "Nolensville", "Oak Hill", "Pegram", "Pleasant View", "Portland", 
          "Ridgetop", "Slayden", "Smyrna", "Spring Hill", "Springfield", "Tennessee Ridge", "Thompson's Station", "Vanleer", "Watertown", "Waverly", 
          "Westmoreland", "White Bluff", "White House"]

In [67]:
test = place.loc[place['Location'].isin(places)]

In [70]:
test.head(100)

,Location,Year,Series,Series Code,Permits
282,Adams,2021,Total Units,1,0
283,Adams,2021,Units in Single-Family Structures,2,0
284,Adams,2021,Units in All Multi-Family Structures,3,0
345,Belle Meade,2021,Total Units,1,14
346,Belle Meade,2021,Units in Single-Family Structures,2,14
347,Belle Meade,2021,Units in All Multi-Family Structures,3,0
351,Berry Hill,2021,Total Units,1,0
352,Berry Hill,2021,Units in Single-Family Structures,2,0
353,Berry Hill,2021,Units in All Multi-Family Structures,3,0
378,Brentwood,2021,Total Units,1,147


In [69]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99 entries, 282 to 1310
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Location     99 non-null     object
 1   Year         99 non-null     object
 2   Series       99 non-null     object
 3   Series Code  99 non-null     object
 4   Permits      99 non-null     object
dtypes: object(5)
memory usage: 4.6+ KB
